# Part 1 of 3

### Import files and connect to page

In [1]:
import pandas as pd
import numpy as np


### Webscape data from wikipedia

In [2]:
import urllib.request
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

soup = BeautifulSoup(page, "lxml")
#all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')


### Loop through correct table and get all data


In [3]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


### once data is retrieved fit into pandas dataframe

In [4]:
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop all Borough with "Not assigned"

In [5]:
df = df[df.Borough != 'Not assigned']
df.head(20)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### Remove special characters and using where choose either borough or neighbourhood

In [6]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df['NewCol'] = np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])
df.head(15)
#df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood,NewCol
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M6A,North York,Lawrence Heights,Lawrence Heights
6,M6A,North York,Lawrence Manor,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park,Queen's Park
9,M9A,Queen's Park,Not assigned,Queen's Park
10,M1B,Scarborough,Rouge,Rouge
11,M1B,Scarborough,Malvern,Malvern
13,M3B,North York,Don Mills North,Don Mills North


### Delete old column, rename new column

In [7]:
df.rename(columns={'Neighbourhood': 'OldCol', 'NewCol': 'Neighbourhood'}, inplace=True)
del df['OldCol']
df[1:10]

,Postcode,Borough,Neighbourhood
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### M9A is now no longer 'Not assigned'

In [8]:
df.shape

(210, 3)

# Part 2 of 3

### Import Geospatial csv

In [9]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
dfCor = pd.read_csv('Geospatial_Coordinates.csv') 

--2020-01-31 04:21:40--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.86, 158.85.108.83
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-31 04:21:43--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-31 04:21:43--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/pub

### Rename column so they can join

In [10]:
dfCor.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [11]:
#dfnew = pd.merge(df,dfCor,by = "Postcode",all.x = TRUE)
dfnew = df.merge(dfCor, on='Postcode')
dfnew.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [12]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

# Part 3 of 3

### Download dependencies

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



### Only boroughs with "Toronto"

In [14]:
dfTor = dfnew[dfnew['Borough'].str.contains("Toronto")]
dfTor.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
35,M4E,East Toronto,The Beaches,43.676357,-79.293031
36,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
40,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
41,M6G,Downtown Toronto,Christie,43.669542,-79.422564
48,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


In [15]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[43.715, -79.38355], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfTor['Latitude'], dfTor['Longitude'], dfTor['Borough'], dfTor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor